# SETUP environment
create a .env file and setup:

* GOOGLE_GENAI_USE_VERTEXAI=FALSE
* GOOGLE_API_KEY=YOUR_KEY

# Install packages:
* google-cloud-aiplatform==1.97.0
* google-adk==1.3.0



# Part 1 - Creating the agent

In [ ]:
# Import classes to define individual and sequential agents using Google's ADK
# LlmAgent: Defines a language model agent with specific instructions
# SequentialAgent: Chains multiple agents in sequence
from google.adk.agents import LlmAgent, SequentialAgent


LLM_MODEL_NAME = "gemini-2.0-flash"


agent_one = LlmAgent(
    model=LLM_MODEL_NAME,
    name="agent_one_name",
    description="Overview description",
    instruction=""" Full prompt of agent one.

    Taks:
    1) Task 1
    2) Task 2
    3) Task 3

    Output:
    Final answer will be store on variable `output_agent_one`
""",
    output_key="output_agent_one"
)


agent_two = LlmAgent(
    model=LLM_MODEL_NAME,
    name="agent_two_name",
    description="Overview description",
    instruction=""" Full prompt of agent two.

    Using the previous output: {output_agent_one} do:

    Taks:
    1) Task 1
    2) Task 2
    3) Task 3

    Output:
    Final answer will be store on variable `output_agent_two`
""",
    output_key="output_agent_two"
)


agent_three = LlmAgent(
    model=LLM_MODEL_NAME,
    name="agent_three_name",
    description="Overview description",
    instruction=""" Full prompt of agent two.

    Using the previous output: {output_agent_two} do:

    Taks:
    1) Task 1
    2) Task 2
    3) Task 3

    Output:
    Final answer will be return to user
"""
)

orchestrator_agent = SequentialAgent(
    name="Name of Final Agent",
    sub_agents=[agent_one, agent_two, agent_three] # The agent will be executed in this order

    description='Overview description',
)


root_agent = orchestrator_agent


# Part 2 - Deploy on Agent Engine

In [ ]:
import os
import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

PROJECT_ID = 'PROJECT ID NUMBER'
LOCATION = 'us-central1'
STAGING_BUCKET = 'gs://BUCKET_NAME'


print('Init VertexAI')
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

app = AdkApp(
    agent=root_agent,
    enable_tracing=True
)

print('Deploy Remote on GCP VertexAI Agent Engine')

remote_app = agent_engines.create(
    app,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ],
    gcs_dir_name=STAGING_BUCKET,
    env_vars={'GOOGLE_API_KEY': os.getenv('GOOGLE_API_KEY'),
                'GOOGLE_GENAI_USE_VERTEXAI': 'TRUE'},
)

print(remote_app.resource_name)


# Part 3 - Call the agent deployed

In [ ]:
from vertexai import agent_engines

AGENT_ID="AGENT ID"
USER_ID="123"
PROJECT_ID = 'PROJECT ID NUMBER'
LOCATION = 'us-central1'
STAGING_BUCKET = 'gs://BUCKET_NAME'


In [ ]:
# Retreive agent from VertexAI Agent Engine
agent = agent_engines.get(f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{AGENT_ID}")

In [ ]:
# Creating new session
remote_session = agent.create_session(user_id=USER_ID)
remote_session

In [ ]:
query_input = "Your query input to agent"

In [ ]:
# Query agent
for event in agent.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message=query_input,
):
    print(event)